# Sequence methods for protein classification

In [44]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

sns.set()

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 8
fig_size[1] = 6
plt.rcParams["figure.figsize"] = fig_size

## Loading and preprocessing the data

In [2]:
data_path = 'data/S.aureus.xlsx'
data = pd.ExcelFile(data_path)
data.sheet_names

['Export Summary', 'Sheet1', 'Sheet2', 'Sheet3']

In [3]:
df1 = data.parse('Sheet1', header=None)
df2 = data.parse('Sheet2', header=None)

In [4]:
print(df1.shape, df2.shape)

(324, 6) (164, 6)


In [5]:
df = pd.concat([df1, df2], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       488 non-null    int64  
 1   1       0 non-null      float64
 2   2       488 non-null    object 
 3   3       366 non-null    object 
 4   4       488 non-null    float64
 5   5       488 non-null    int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 23.0+ KB


In [6]:
df.drop_duplicates(inplace=True)
df.head()

,0,1,2,3,4,5
0,7666,NaN,WKRLVAASAKKK,AMD,10.00,1
1,10153,NaN,FKAWRWAWRMKKLAAPS,NaN,8.00,1
2,14883,NaN,GLFDIIAKIAESF,AMD,512.00,0
3,13995,NaN,FLGALWHALSHLL,NaN,11.82,1
4,7083,NaN,RRWKIVVIRWRR,NaN,1.21,1


In [7]:
df.drop([0, 1, 4], axis=1, inplace=True)

In [8]:
df

,2,3,5
0,WKRLVAASAKKK,AMD,1
1,FKAWRWAWRMKKLAAPS,NaN,1
2,GLFDIIAKIAESF,AMD,0
3,FLGALWHALSHLL,NaN,1
4,RRWKIVVIRWRR,NaN,1
...,...,...,...
483,AAGMGFFGAR,AMD,1
484,FPVTWPTKWWEG,AMD,0
485,MWITNGGVANWYFVLAR,NaN,1
486,LLRRVAGLLKQFAK,NaN,1


In [9]:
df[2] = ('X' + df[2]).where(df[3]=='AMD', df[2])
df.drop(3, axis=1, inplace=True)

In [10]:
df

,2,5
0,XWKRLVAASAKKK,1
1,FKAWRWAWRMKKLAAPS,1
2,XGLFDIIAKIAESF,0
3,FLGALWHALSHLL,1
4,RRWKIVVIRWRR,1
...,...,...
483,XAAGMGFFGAR,1
484,XFPVTWPTKWWEG,0
485,MWITNGGVANWYFVLAR,1
486,LLRRVAGLLKQFAK,1


## Analysis

In [ ]:
# we will take each unique character and count how many times it enters a sequence in the positive and negative classes

In [11]:
def process_seq(seq):
    return [char for char in seq]

In [12]:
def build_freqs(seqs, ys):
    freqs = {}
    for y, seq in zip(ys, seqs):
        for char in process_seq(seq):
            pair = (char, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs
        

In [13]:
seqs = df[2].tolist()
ys = df[5].tolist()
print(len(seqs), len(ys))

488 488


In [14]:
freqs = build_freqs(seqs, ys)
freqs

{('X', 1): 256,
 ('W', 1): 403,
 ('K', 1): 638,
 ('R', 1): 635,
 ('L', 1): 713,
 ('V', 1): 239,
 ('A', 1): 251,
 ('S', 1): 114,
 ('F', 1): 278,
 ('M', 1): 26,
 ('P', 1): 67,
 ('X', 0): 109,
 ('G', 0): 149,
 ('L', 0): 217,
 ('F', 0): 154,
 ('D', 0): 38,
 ('I', 0): 174,
 ('A', 0): 128,
 ('K', 0): 329,
 ('E', 0): 28,
 ('S', 0): 95,
 ('G', 1): 255,
 ('H', 1): 40,
 ('I', 1): 341,
 ('N', 1): 50,
 ('R', 0): 292,
 ('T', 0): 41,
 ('P', 0): 120,
 ('C', 0): 16,
 ('W', 0): 108,
 ('V', 0): 107,
 ('N', 0): 18,
 ('M', 0): 19,
 ('Q', 0): 30,
 ('Y', 1): 38,
 ('H', 0): 39,
 ('Q', 1): 29,
 ('Y', 0): 40,
 ('T', 1): 59,
 ('D', 1): 4,
 ('C', 1): 5,
 ('E', 1): 7}

In [15]:
len(freqs)

42

In [146]:
# we now extract features for each sequence
def extract_features(seq, freqs):
    
    char_list = process_seq(seq)
    
    # define the feature vector, including unit bias term
    x = np.zeros((1, 3))
    x[0, 0] = 1
    
    for char in char_list:
        x[0, 1] += freqs.get((char, 1.), 0)
        x[0, 2] += freqs.get((char, 0.), 0)
    
    assert(x.shape == (1,3))
    return x

In [151]:
# construct the full feature matrix
X = np.zeros((len(seqs), 3))
X.shape

(488, 3)

In [152]:
for i in range(len(seqs)):
    X[i, :] = extract_features(seqs[i], freqs)
y = ys

In [157]:
X = np.asarray(X)
y = np.asarray(y)
print(X.shape, y.shape)

(488, 3) (488,)


In [194]:
# fitting logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

log_reg = LogisticRegression(C = 10000)

In [189]:
log_reg.classes_

array([0, 1])

In [197]:
cross_val_score(log_reg, X, y, cv=5, scoring="accuracy").mean()

0.7888701872501578

In [191]:
print(confusion_matrix(y, y_pred))

[[ 85  79]
 [ 24 300]]


# Naive Bayes

In [39]:
seqs = df[2].tolist()
ys = df[5].tolist()
print(len(seqs), len(ys))

488 488


In [40]:
def process_seq(seq):
    return [char for char in seq]

In [41]:
# function to count occurence of each character in +/- seqs
def update_count(result, seqs, ys):
    
    for y, seq in zip(ys, seqs):
        
        for char in process_seq(seq):
        
            pair = (char, y)
        
            if pair in result:
                result[pair] += 1
        
            else:
                result[pair] = 1
    
    return result

In [48]:
freqs = update_count({}, seqs, ys)
freqs

{('X', 1): 256,
 ('W', 1): 403,
 ('K', 1): 638,
 ('R', 1): 635,
 ('L', 1): 713,
 ('V', 1): 239,
 ('A', 1): 251,
 ('S', 1): 114,
 ('F', 1): 278,
 ('M', 1): 26,
 ('P', 1): 67,
 ('X', 0): 109,
 ('G', 0): 149,
 ('L', 0): 217,
 ('F', 0): 154,
 ('D', 0): 38,
 ('I', 0): 174,
 ('A', 0): 128,
 ('K', 0): 329,
 ('E', 0): 28,
 ('S', 0): 95,
 ('G', 1): 255,
 ('H', 1): 40,
 ('I', 1): 341,
 ('N', 1): 50,
 ('R', 0): 292,
 ('T', 0): 41,
 ('P', 0): 120,
 ('C', 0): 16,
 ('W', 0): 108,
 ('V', 0): 107,
 ('N', 0): 18,
 ('M', 0): 19,
 ('Q', 0): 30,
 ('Y', 1): 38,
 ('H', 0): 39,
 ('Q', 1): 29,
 ('Y', 0): 40,
 ('T', 1): 59,
 ('D', 1): 4,
 ('C', 1): 5,
 ('E', 1): 7}

In [53]:
def train_naive_bayes(freqs, train_x, train_y):
    
    loglikelihood = {}
    logprior = 0
    
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    
    N_pos = N_neg = 0
    for pair in freqs.keys():
        if pair[1] > 0:
            N_pos += freqs[pair]
        else:
            N_neg += freqs[pair]
    
    D = len(train_y)
    D_pos = np.sum(np.array(train_y))
    D_neg = D - D_pos
    
    logprior = np.log(D_pos) - np.log(D_neg)
    
    for char in vocab:
        freq_pos = freqs.get((char, 1.), 0)
        freq_neg = freqs.get((char, 0.), 0)
        
        p_c_pos = (freq_pos + 1) / (N_pos + V)
        p_c_neg = (freq_neg + 1) / (N_neg + V)
        
        loglikelihood[char] = np.log(p_c_pos) - np.log(p_c_neg)
    
    return logprior, loglikelihood

In [54]:
logprior, loglikelihood = train_naive_bayes(freqs, seqs, ys)

In [55]:
logprior

0.6808770879681303

In [56]:
loglikelihood

{'C': -1.7179580439205697,
 'G': -0.14196201870910263,
 'I': -0.006479405953318196,
 'S': -0.49592023219699577,
 'N': 0.3108824844654752,
 'R': 0.09852178523064548,
 'T': -0.3198292251536774,
 'Q': -0.7092939919154002,
 'D': -2.7306279027879556,
 'X': 0.1720915500103941,
 'F': -0.08871750419029079,
 'M': -0.37639957664207113,
 'V': 0.12200352712536189,
 'H': -0.6518115565020377,
 'P': -1.2527870095130433,
 'A': -0.006887485942657978,
 'W': 0.6335628266395967,
 'E': -1.9643584573990474,
 'Y': -0.7265145896670706,
 'K': -0.015692369175400733,
 'L': 0.5098837304580861}

In [65]:
def naive_bayes_predict(seq, logprior, loglikelihood):
    
    char_list = process_seq(seq)
    
    p = 0
    
    p += logprior
    
    for char in char_list:
        if char in loglikelihood:
            p += loglikelihood[char]
    
    return p

In [66]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    
    accuracy = 0
    
    y_hats = []
    
    for seq in test_x:
        if naive_bayes_predict(seq, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)
        
    error = np.mean(np.array(test_y) != np.array(y_hats))
    
    accuracy = 1 - error
    
    return accuracy

In [67]:
test_naive_bayes(seqs, ys, logprior, loglikelihood)

0.7602459016393442

In [74]:
# break into train and test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(seqs), np.array(ys), test_size=0.2)

In [75]:
logprior, loglikelihood = train_naive_bayes(freqs, X_train, y_train)
logprior, loglikelihood

(0.7163142398414797,
 {'C': -1.7179580439205697,
  'G': -0.14196201870910263,
  'I': -0.006479405953318196,
  'S': -0.49592023219699577,
  'N': 0.3108824844654752,
  'R': 0.09852178523064548,
  'T': -0.3198292251536774,
  'Q': -0.7092939919154002,
  'D': -2.7306279027879556,
  'X': 0.1720915500103941,
  'F': -0.08871750419029079,
  'M': -0.37639957664207113,
  'V': 0.12200352712536189,
  'H': -0.6518115565020377,
  'P': -1.2527870095130433,
  'A': -0.006887485942657978,
  'W': 0.6335628266395967,
  'E': -1.9643584573990474,
  'Y': -0.7265145896670706,
  'K': -0.015692369175400733,
  'L': 0.5098837304580861})

In [76]:
test_naive_bayes(X_test, y_test, logprior, loglikelihood)

0.7653061224489796

## A bigger dataset with logistic regression

In [79]:
import pandas as pd

In [80]:
df = pd.read_csv('data/bigger_data.csv')

In [81]:
df.head()

,ID,SEQUENCE,C TERMINUS,ACTIVITY,Unnamed: 4
0,11866,GIKKWVKGVAKGVAKDLAKKIL,AMD,0.59,1.0
1,10928,GKPRPYPPRPPPHPRPIRV,NaN,0.66,1.0
2,14031,WLKAAAKAAAKLW,AMD,0.71,1.0
3,6841,RRLFRRILRWL,AMD,0.79,1.0
4,12612,KKWRKWLKWLAKK,NaN,0.80,1.0


In [82]:
df.columns = ['ID', 'seq', 'c_term', 'activity', 'label']
df.head()

,ID,seq,c_term,activity,label
0,11866,GIKKWVKGVAKGVAKDLAKKIL,AMD,0.59,1.0
1,10928,GKPRPYPPRPPPHPRPIRV,NaN,0.66,1.0
2,14031,WLKAAAKAAAKLW,AMD,0.71,1.0
3,6841,RRLFRRILRWL,AMD,0.79,1.0
4,12612,KKWRKWLKWLAKK,NaN,0.80,1.0


In [83]:
df.drop(['ID', 'activity'], axis=1, inplace=True)

In [84]:
df.dtypes

seq        object
c_term     object
label     float64
dtype: object

In [85]:
df.head()

,seq,c_term,label
0,GIKKWVKGVAKGVAKDLAKKIL,AMD,1.0
1,GKPRPYPPRPPPHPRPIRV,NaN,1.0
2,WLKAAAKAAAKLW,AMD,1.0
3,RRLFRRILRWL,AMD,1.0
4,KKWRKWLKWLAKK,NaN,1.0


In [86]:
df['seq'] = ('X' + df['seq']).where(df['c_term']=='AMD', df['seq'])

In [87]:
df.head()

,seq,c_term,label
0,XGIKKWVKGVAKGVAKDLAKKIL,AMD,1.0
1,GKPRPYPPRPPPHPRPIRV,NaN,1.0
2,XWLKAAAKAAAKLW,AMD,1.0
3,XRRLFRRILRWL,AMD,1.0
4,KKWRKWLKWLAKK,NaN,1.0


In [88]:
df.drop(['c_term'], axis=1, inplace=True)

In [89]:
df.head()

,seq,label
0,XGIKKWVKGVAKGVAKDLAKKIL,1.0
1,GKPRPYPPRPPPHPRPIRV,1.0
2,XWLKAAAKAAAKLW,1.0
3,XRRLFRRILRWL,1.0
4,KKWRKWLKWLAKK,1.0


In [90]:
df.dropna(inplace=True)
len(df)

1147

In [91]:
def process_seq(seq):
    return [char for char in seq]

In [92]:
def build_freqs(seqs, ys):
    freqs = {}
    for y, seq in zip(ys, seqs):
        for char in process_seq(seq):
            pair = (char, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [93]:
seqs = df['seq'].tolist()
ys = df['label'].tolist()
print(len(seqs), len(ys))

1147 1147


In [94]:
freqs = build_freqs(seqs, ys)
freqs

{('X', 1.0): 499,
 ('G', 1.0): 821,
 ('I', 1.0): 911,
 ('K', 1.0): 2002,
 ('W', 1.0): 775,
 ('V', 1.0): 686,
 ('A', 1.0): 818,
 ('D', 1.0): 78,
 ('L', 1.0): 1538,
 ('P', 1.0): 400,
 ('R', 1.0): 1621,
 ('Y', 1.0): 133,
 ('H', 1.0): 223,
 ('F', 1.0): 723,
 ('S', 1.0): 240,
 ('Q', 1.0): 151,
 ('N', 1.0): 189,
 ('C', 1.0): 53,
 ('T', 1.0): 205,
 ('M', 1.0): 85,
 ('E', 1.0): 51,
 ('G', 0.0): 615,
 ('W', 0.0): 204,
 ('A', 0.0): 514,
 ('N', 0.0): 157,
 ('T', 0.0): 128,
 ('L', 0.0): 887,
 ('K', 0.0): 954,
 ('V', 0.0): 331,
 ('C', 0.0): 31,
 ('I', 0.0): 596,
 ('X', 0.0): 293,
 ('S', 0.0): 264,
 ('M', 0.0): 103,
 ('E', 0.0): 104,
 ('Y', 0.0): 85,
 ('Q', 0.0): 98,
 ('R', 0.0): 542,
 ('H', 0.0): 161,
 ('F', 0.0): 355,
 ('P', 0.0): 329,
 ('D', 0.0): 142}

In [95]:
len(freqs)

42

In [96]:
# we now extract features for each sequence
def extract_features(seq, freqs):
    
    char_list = process_seq(seq)
    
    # define the feature vector, including unit bias term
    x = np.zeros((1, 3))
    x[0, 0] = 1
    
    for char in char_list:
        x[0, 1] += freqs.get((char, 1.), 0)
        x[0, 2] += freqs.get((char, 0.), 0)
    
    assert(x.shape == (1,3))
    return x

In [97]:
# construct the full feature matrix
X = np.zeros((len(seqs), 3))
X.shape

(1147, 3)

In [98]:
for i in range(len(seqs)):
    X[i, :] = extract_features(seqs[i], freqs)
y = ys

In [99]:
X = np.asarray(X)
y = np.asarray(y)
print(X.shape, y.shape)

(1147, 3) (1147,)


In [103]:
# fitting logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

log_reg = LogisticRegression(C = 10000)

In [101]:
cross_val_score(log_reg, X, y, cv=5, scoring="accuracy").mean()

0.7000341750522118

In [104]:
log_reg.fit(X, y)

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
log_reg.classes_

array([0., 1.])

In [107]:
np.mean(log_reg.predict(X) == y)

0.7079337401918047

## A bigger dataset with Naive Bayes

In [122]:
import pandas as pd
df = pd.read_csv('data/bigger_data.csv')
df.head()

,ID,SEQUENCE,C TERMINUS,ACTIVITY,Unnamed: 4
0,11866,GIKKWVKGVAKGVAKDLAKKIL,AMD,0.59,1.0
1,10928,GKPRPYPPRPPPHPRPIRV,NaN,0.66,1.0
2,14031,WLKAAAKAAAKLW,AMD,0.71,1.0
3,6841,RRLFRRILRWL,AMD,0.79,1.0
4,12612,KKWRKWLKWLAKK,NaN,0.80,1.0


In [123]:
df.columns = ['ID', 'seq', 'c_term', 'activity', 'label']
df.drop(['ID', 'activity'], axis=1, inplace=True)
df['seq'] = ('X' + df['seq']).where(df['c_term']=='AMD', df['seq'])
df.drop(['c_term'], axis=1, inplace=True)
df.dropna(inplace=True)

In [125]:
df.head(10)

,seq,label
0,XGIKKWVKGVAKGVAKDLAKKIL,1.0
1,GKPRPYPPRPPPHPRPIRV,1.0
2,XWLKAAAKAAAKLW,1.0
3,XRRLFRRILRWL,1.0
4,KKWRKWLKWLAKK,1.0
5,SKFFRKARKKLGKGLQKIKNVLRKY,1.0
6,GKPRPYSPRPSSHPRPIRV,1.0
7,XGFKLGRKLVKVFKWII,1.0
8,KRIVQRIKKFLR,1.0
9,GSKKPVPIIYCNRRTGKCQRM,1.0


In [113]:
seqs = df['seq'].tolist()
ys = df['label'].tolist()
print(len(seqs), len(ys))

1147 1147


In [114]:
def process_seq(seq):
    return [char for char in seq]

# function to count occurence of each character in +/- seqs
def update_count(result, seqs, ys):
    
    for y, seq in zip(ys, seqs):
        
        for char in process_seq(seq):
        
            pair = (char, y)
        
            if pair in result:
                result[pair] += 1
        
            else:
                result[pair] = 1
    
    return result

freqs = update_count({}, seqs, ys)
freqs

{('X', 1.0): 499,
 ('G', 1.0): 821,
 ('I', 1.0): 911,
 ('K', 1.0): 2002,
 ('W', 1.0): 775,
 ('V', 1.0): 686,
 ('A', 1.0): 818,
 ('D', 1.0): 78,
 ('L', 1.0): 1538,
 ('P', 1.0): 400,
 ('R', 1.0): 1621,
 ('Y', 1.0): 133,
 ('H', 1.0): 223,
 ('F', 1.0): 723,
 ('S', 1.0): 240,
 ('Q', 1.0): 151,
 ('N', 1.0): 189,
 ('C', 1.0): 53,
 ('T', 1.0): 205,
 ('M', 1.0): 85,
 ('E', 1.0): 51,
 ('G', 0.0): 615,
 ('W', 0.0): 204,
 ('A', 0.0): 514,
 ('N', 0.0): 157,
 ('T', 0.0): 128,
 ('L', 0.0): 887,
 ('K', 0.0): 954,
 ('V', 0.0): 331,
 ('C', 0.0): 31,
 ('I', 0.0): 596,
 ('X', 0.0): 293,
 ('S', 0.0): 264,
 ('M', 0.0): 103,
 ('E', 0.0): 104,
 ('Y', 0.0): 85,
 ('Q', 0.0): 98,
 ('R', 0.0): 542,
 ('H', 0.0): 161,
 ('F', 0.0): 355,
 ('P', 0.0): 329,
 ('D', 0.0): 142}

In [115]:
def train_naive_bayes(freqs, train_x, train_y):
    
    loglikelihood = {}
    logprior = 0
    
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    
    N_pos = N_neg = 0
    for pair in freqs.keys():
        if pair[1] > 0:
            N_pos += freqs[pair]
        else:
            N_neg += freqs[pair]
    
    D = len(train_y)
    D_pos = np.sum(np.array(train_y))
    D_neg = D - D_pos
    
    logprior = np.log(D_pos) - np.log(D_neg)
    
    for char in vocab:
        freq_pos = freqs.get((char, 1.), 0)
        freq_neg = freqs.get((char, 0.), 0)
        
        p_c_pos = (freq_pos + 1) / (N_pos + V)
        p_c_neg = (freq_neg + 1) / (N_neg + V)
        
        loglikelihood[char] = np.log(p_c_pos) - np.log(p_c_neg)
    
    return logprior, loglikelihood

In [116]:
logprior, loglikelihood = train_naive_bayes(freqs, seqs, ys)

In [118]:
loglikelihood

{'H': -0.24572136467303718,
 'C': -0.04652293753114556,
 'E': -1.272487712871789,
 'D': -1.163167859088579,
 'N': -0.3853420421621738,
 'F': 0.1400895802379929,
 'Q': -0.14101041058400643,
 'K': 0.17091891388939495,
 'X': -0.03874275021218265,
 'L': -0.01985469044898336,
 'I': -0.1460482046139635,
 'S': -0.6647039737912603,
 'T': -0.10170731686778423,
 'V': 0.1574282420101678,
 'A': -0.1058538981063597,
 'M': -0.7598146841835574,
 'W': 0.7613714597491175,
 'P': -0.37490230844964945,
 'Y': -0.1262785775982893,
 'G': -0.28127764977303293,
 'R': 0.5245348334457298}

In [119]:
def naive_bayes_predict(seq, logprior, loglikelihood):
    
    char_list = process_seq(seq)
    
    p = 0
    
    p += logprior
    
    for char in char_list:
        if char in loglikelihood:
            p += loglikelihood[char]
    
    return p

In [120]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    
    accuracy = 0
    
    y_hats = []
    
    for seq in test_x:
        if naive_bayes_predict(seq, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)
        
    error = np.mean(np.array(test_y) != np.array(y_hats))
    
    accuracy = 1 - error
    
    return accuracy

In [121]:
test_naive_bayes(seqs, ys, logprior, loglikelihood)

0.6756756756756757